In [1]:
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints._base import Endpoint
from nba_api.stats.library.http import NBAStatsHTTP
from nba_api.stats.endpoints import playerdashboardbyteamperformance
from nba_api.stats.endpoints import leaguedashplayerstats
from nba_api.stats.endpoints import teamdashboardbyteamperformance
from nba_api.stats.endpoints import playerdashboardbylastngames
from nba_api.stats.endpoints import teamdashboardbylastngames
from nba_api.stats.endpoints import teamplayerdashboard
from nba_api.stats.static import teams
from nba_api.stats.static import players
import pandas
import inspect

In [2]:
#Player Input

player1_input = input('Enter player 1: ')
print(player1_input)

player1 = players.find_players_by_full_name(player1_input)[0]['id']
player1_name = players.find_players_by_full_name(player1_input)[0]['full_name']

player_info = playerdashboardbylastngames.PlayerDashboardByLastNGames(player_id=player1)

# player_headers = player_info.overall_player_dashboard.get_dict().get('headers')
# player_data = player_info.overall_player_dashboard.get_dict().get('data')[0]

##last 20 games
player_headers = player_info.last20_player_dashboard.get_dict().get('headers')
player_data = player_info.last20_player_dashboard.get_dict().get('data')[0]

print(player_headers[5], player_data[5], 
	player_headers[2], player_data[2], 
	player_headers[26], player_data[26],
	player_headers[27], player_data[27])

Enter player 1: alex caruso
alex caruso


IndexError: list index out of range

In [5]:
#Team Input

team1_input = input('Enter team 1: ')
print(team1_input)
team1 = teams.find_teams_by_full_name(team1_input)[0]['id']

teamdashboard = teamdashboardbylastngames.TeamDashboardByLastNGames(team_id=team1)
teamdashboard_adv = teamdashboardbylastngames.TeamDashboardByLastNGames(team_id=team1, 
                                                                    measure_type_detailed_defense='Advanced')

# team_headers = teamdashboard.overall_team_dashboard.get_dict().get('headers')
# team_data = teamdashboard.overall_team_dashboard.get_dict().get('data')[0]

##last 20 games
team_headers_basic = teamdashboard.last20_team_dashboard.get_dict().get('headers')
team_data_basic = teamdashboard.last20_team_dashboard.get_dict().get('data')[0]
team_headers = teamdashboard_adv.last20_team_dashboard.get_dict().get('headers')
team_data = teamdashboard_adv.last20_team_dashboard.get_dict().get('data')[0]

print(team1, team_headers[23], team_data[23], team_headers[10], team_data[10], 
      team_headers[2], team_data[2], team_headers[3], team_data[3], 
      team_headers[4], team_data[4], team_headers_basic[9], team_data_basic[9])

Enter team 1: lakers
lakers


IndexError: list index out of range

In [58]:
###PER stats (2018-19)####
lg_AST = 24.6
lg_FG = 41.1
lg_PTS = 111.2
lg_FGA = 89.2
lg_FT = 17.7
lg_FTA = 23.1
lg_TRB = 45.2
lg_ORB = 10.3
lg_TOV = 14.1
lg_PF = 20.9
lg_pace = 100

factor = (2/3) - (0.5*(lg_AST/lg_FG)) / (2*(lg_FG/lg_FT)) 
VOP = lg_PTS / (lg_FGA-lg_ORB+lg_TOV+0.44*lg_FTA) 
DRB_perc = (lg_TRB -lg_ORB) / lg_TRB
 
print('league stats:', factor, VOP, DRB_perc)

#####PER player stats####
MP = player_data[6]
ThrP = player_data[10] 
AST = player_data[19]
TOV = player_data[20]
FGA = player_data[8]
FG = player_data[7] + player_data[10] 
FTA = player_data[14]
FT = player_data[13]
TRB = player_data[18]
ORB = player_data[16]
STL = player_data[21]
BLK = player_data[22]
PF = player_data[24]
print(player1, 'mp', MP)

####PER team stats####
team_AST = team_data_basic[19]
team_FG = team_data_basic[7]+team_data_basic[10]
team_pace = team_data[23] 
print('team ast', team_AST, 'team FG', team_FG, 'team_pace', team_pace)

league stats: 0.6022253005842968 1.0778953898646815 0.7721238938053098
1627936 mp 511.5366666666667
team ast 528 team FG 1041 team_pace 102.53


In [59]:
####PER Calculation####                    
PER_calc_player = (1 / MP) * ( ThrP + (2/3) * AST
     + (2 - factor * (team_AST / team_FG)) * FG
     + (FT *0.5 * (1 + (1 - (team_AST / team_FG)) + (2/3) * (team_AST / team_FG)))
     - VOP * TOV - VOP * DRB_perc * (FGA - FG)
     - VOP * 0.44 * (0.44 + (0.56 * DRB_perc)) * (FTA - FT)
     + VOP * (1 - DRB_perc) * (TRB - ORB) + VOP * DRB_perc * ORB + VOP * STL + VOP * DRB_perc * BLK - PF * ((lg_FT / lg_PF) - 0.44 * (lg_FTA / lg_PF) * VOP) )

adj_PER_calc_player = PER_calc_player * (lg_pace /team_pace)

print(PER_calc_player, adj_PER_calc_player)

0.4292969468108909 0.4187037421348785


In [60]:
#Individual Player Defense Stats

player_info_def = leaguedashplayerstats.LeagueDashPlayerStats(last_n_games=20, measure_type_detailed_defense='Defense')

player_def_headers = player_info_def.league_dash_player_stats.get_dict().get('headers')
player_def_data = player_info_def.league_dash_player_stats.get_dict().get('data')

#print('headers:', player_def_headers)
#print('data:', player_def_data)
for n in player_def_data:
    if player1 in n:
        print(n)


[1627936, 'Alex Caruso', 1610612747, 'LAL', 25.0, 19, 7, 12, 0.368, 509.88666666666666, 103.2, 47, 0.083, 0.125, 24, 0.279, 7, 0.132, 199, 149, 161, 470, 0.19, 104, 197, 398, 316, 99, 82, 187, 379, 366, 28, 90, 130, 233, 429, 392, 396, 333, 28, 5, '1627936,1610612747']


In [61]:
roster1 = teamplayerdashboard.TeamPlayerDashboard(team_id=team1, last_n_games=20).get_dict()#.PlayersSeasonTotals(team_id=team1, last_n_games=20)
team1_roster_last20 = []

#print(roster1)['resultSets'][1])#['rowSet'])
for n in roster1['resultSets'][1]['rowSet']:
    print(n[2], 'GP:', n[3], 'id:', n[1], 'Min', n[7])
    team1_roster_last20.append(n[2])
    
print(team1_roster_last20)

Alex Caruso GP: 19 id: 1627936 Min 509.88666666666666
Andre Ingram GP: 4 id: 201281 Min 14.626666666666667
Brandon Ingram GP: 1 id: 1627742 Min 39.1
Isaac Bonga GP: 14 id: 1629067 Min 89.50833333333334
JaVale McGee GP: 20 id: 201580 Min 510.68833333333333
Jemerrio Jones GP: 6 id: 1629203 Min 143.11666666666667
Johnathan Williams GP: 17 id: 1629140 Min 282.28333333333336
Josh Hart GP: 8 id: 1628404 Min 211.65166666666667
Kentavious Caldwell-Pope GP: 20 id: 203484 Min 582.175
Kyle Kuzma GP: 12 id: 1628398 Min 408.99666666666667
Lance Stephenson GP: 8 id: 202362 Min 180.09166666666667
LeBron James GP: 11 id: 2544 Min 380.05833333333334
Mike Muscala GP: 13 id: 203488 Min 221.27666666666667
Moritz Wagner GP: 20 id: 1629021 Min 292.24
Rajon Rondo GP: 18 id: 200765 Min 583.9866666666667
Reggie Bullock GP: 12 id: 203493 Min 297.1816666666667
Scott Machado GP: 4 id: 203159 Min 19.166666666666668
Tyson Chandler GP: 3 id: 2199 Min 33.965
['Alex Caruso', 'Andre Ingram', 'Brandon Ingram', 'Isaac Bo

In [ ]:
#Theory: If second game of back-to-back is a road game, odds of winning decrease 7-8%, multiply ratings by 0.93.